<a href="https://colab.research.google.com/github/AstridXiomara/LSTM/blob/main/LocalSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LIBRERIAS

In [5]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import datetime
import math

In [6]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.metrics import mean_squared_error


## Funciones LSTM

### LSTM

#### Preparar Data

In [ ]:
# Divide el dataset en subsets train/test.
def train_test_split(data, n_test):
    return data[:-n_test,:], data[-n_test:,:]

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

#### Vanilla, Stacked, Bidirectional, Encoder

In [ ]:
# Modelo

def modelo(data, parametros):
  
  LSTM_model, nodes, activacion, n_steps_in, n_features, n_steps_out, optimizador, perdidas, epocas, n_test=parametros
  
  train, test = train_test_split(data, n_test)
  x_train, y_train = split_sequences(train, n_steps_in,n_steps_out)
  x_test, y_test = split_sequences(test, n_steps_in,n_steps_out)
  
  #modelo
  if LSTM_model=="Vanilla":
    # define model
    model = Sequential()
    #Number of LSTM in hidden layer=50
    model.add(LSTM(nodes, activation=activacion, input_shape=(n_steps_in, n_features)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer=optimizador, loss=perdidas)
  elif LSTM_model=="Stacked":
    # define model
    model = Sequential()
    model.add(LSTM(nodes, activation=activacion, return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(nodes, activation=activacion))
    model.add(Dense(n_steps_out))
    model.compile(optimizer=optimizador, loss=perdidas)
  elif LSTM_model=="Bidirectional":
    # define model
    model = Sequential()
    model.add(Bidirectional(LSTM(nodes, activation=activacion), input_shape=(n_steps_in, n_features)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer=optimizador, loss=perdidas)
  elif LSTM_model=="Encoder-Decoder":
    model = Sequential()
    model.add(LSTM(nodes, activation=activacion, input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(nodes, activation=activacion, return_sequences=True))
    model.add(TimeDistributed(Dense(n_steps_out)))
    model.compile(optimizer=optimizador, loss=perdidas)
  # fit model
  model.fit(x_train, y_train, epochs=epocas, verbose=0)

  #Prediccion
  yhat=np.zeros(len(y_test))
  for i in range (0, len(y_test)):
    # demonstrate prediction
    x_input = array(x_test[i,:,:])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat[i] = model.predict(x_input, verbose=0)

  RMSE = math.sqrt(mean_squared_error(y_test, yhat))
  return y_test,yhat, RMSE

#### CNN LSTM

In [ ]:
#CNN
def CNN_LSTM(data, parametros):

  LSTM_model, nodes, activacion, n_steps_in, n_features, n_steps_out, optimizador, perdidas, epocas, n_test, n_seq, n_steps, filtros, kernelsize =parametros
  
  train, test = train_test_split(data, n_test)
  x_train, y_train = split_sequences(train, n_steps_in, n_steps_out)
  x_test, y_test = split_sequences(test, n_steps_in, n_steps_out)
  
  X = x_train.reshape((x_train.shape[0], n_seq, n_steps, n_features))

  # define model
  model = Sequential()
  model.add(TimeDistributed(Conv1D(filters=filtros, kernel_size=kernelsize, activation=activacion), input_shape=(None, n_steps, n_features)))
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  if LSTM_model=="Vanilla":
    # define model
    #Number of LSTM in hidden layer=50
    model.add(LSTM(nodes, activation=activacion))
    model.add(Dense(n_steps_out))
    model.compile(optimizer=optimizador, loss=perdidas)
  elif LSTM_model=="Stacked":
    # define model
    model.add(LSTM(nodes, activation=activacion, return_sequences=True))
    model.add(LSTM(nodes, activation=activacion))
    model.add(Dense(n_steps_out))
    model.compile(optimizer=optimizador, loss=perdidas)
  elif LSTM_model=="Bidirectional":
    # define model
    model.add(Bidirectional(LSTM(nodes, activation=activacion)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer=optimizador, loss=perdidas)

  # fit model
  model.fit(X, y_train, epochs=epocas, verbose=0)

  X2 = x_test.reshape((x_test.shape[0], n_seq, n_steps, n_features))
  yhat=np.zeros(len(y_test))
  for i in range (0, len(y_test)):
    x_input = array(X2[i,:,:])
    x_input = x_input.reshape((1, n_seq, n_steps, n_features))
    yhat[i] = model.predict(x_input, verbose=0)
  
  RMSE=math.sqrt(mean_squared_error(y_test, yhat))
  #RMSE = math.sqrt(mean_squared_error( y_test, yhat))
  return (y_test, yhat, RMSE)


#### Conv LSTM

In [ ]:
def Conv_LSTM(data, parametros):
  
  nodes, activacion, n_steps_in, n_features, n_steps_out, optimizador, perdidas, epocas, n_test, n_seq, n_steps, filtros=parametros
  
  train, test = train_test_split(data, n_test)
  x_train, y_train = split_sequences(train, n_steps_in, n_steps_out)
  x_test, y_test = split_sequences(test, n_steps_in, n_steps_out)
  
  X = x_train.reshape((x_train.shape[0], n_seq, 1, n_steps, n_features))

  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=filtros, kernel_size=(1,2), activation=activacion, input_shape=(n_seq, 1, n_steps, n_features)))
  model.add(Flatten())
  model.add(Dense(n_steps_out))
  model.compile(optimizer=optimizador, loss=perdidas)

  # fit model
  model.fit(X, y_train, epochs=epocas, verbose=0)

  yhat=np.zeros(len(y_test))
  for i in range (0, len(y_test)):
    x_input = array(x_test[i,:,:])
    x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
    yhat[i] = model.predict(x_input, verbose=0)
  
  RMSE=math.sqrt(mean_squared_error(y_test, yhat))
  return (y_test, yhat, RMSE)


### VPN

In [ ]:
def VPN(qo,qw,Qw,t, wells, PO, PE, Transp, LC, WD, WT, CE, TRM, Taxes, Royal, TEA):
    oil=np.zeros(t)
    water=np.zeros(t)
    income=np.zeros(t)
    cost=np.zeros(t)
    gross=np.zeros(t)
    net=np.zeros(t)
    Valor=np.zeros(t)
    NPV=0
    for j in range(0,t):
        for i in range[0,wells]:
            oil[j]+=qo[j,i]
            water[j]+=qw[j,i]
        income[j]= oil[j]*PO[j]*15
        cost[j]  = (oil[j]*(LC+Transp)+
                    water[j]*WD +
                    Qw[j]*WT+
                    CE*PE[j]*24/TRM)*15
        gross[j]=(income[j]-cost[j])*[1-Royal]
        net[j]=gross[j]*(1-Taxes)
        Valor[j]=net[j]*np.exp(-TEA*j/24)
        NPV+=Valor[j]
    return NPV

### Local Search

In [ ]:
def LocalSearch(Presion, Qw, Wells, t, DeltaP, DeltaQ):
  qo,qw=LSTM(Presion, Qw, Wells, t)

In [ ]:
##Optimos
Oil_well_1="Encoder-Decoder",          209, "linear", 7, 5, 1, "adam", "mse", 460,24
y_test, y1, RMSE1=modelo(data_interes, parametros1)

parametros5= "Vanilla",         272, "sigmoid", 8, 5, 1, "adam", "mse", 280,24, 2, 4, 87, 1
y_test, y5, RMSE5=CNN_LSTM(data_interes, parametros5)

parametros8=                    257, "sigmoid", 8, 5, 1, "adam", "mse", 447,24, 2, 4, 175
y_test, y8, RMSE8=Conv_LSTM(data_interes, parametros8)

y_test, y1, RMSE1, y5, RMSE5, y8, RMSE8